In [ ]:
import pandas as pd


In [ ]:

data = pd.read_csv('/content/Private AI principles & Abstract Scores.csv',encoding= 'unicode_escape')
patent=data
patent

,CID,Company,Principles,Year,Source,TotalAIEthicsPub,TotalAIpub,AbstractScore
0,291,3M,David Frazee is vice president of 3Ms CRSL gr...,2022,https://futures.3m.com/Artificial-Intelligence...,NaN,82.0,NaN
1,67,Accenture,We welcome steps already being taken in this s...,2019,http://www.ethicscodescollection.org/detail/ae...,8.0,250.0,0.0
2,1451,Adobe,Adobe is taking a deliberate and thoughtful ap...,2022,AI Ethics (adobe.com),2.0,927.0,2.0
3,1693,Amazon,"Responsible Use of Machine Learning \nAt AWS, ...",2022,https://aws.amazon.com/machine-learning/respon...,21.0,710.0,0.0
4,234,American Express,Artificial Intelligence (AI) offers companies ...,2018,https://www.americanexpress.com/en-us/business...,NaN,7.0,NaN
...,...,...,...,...,...,...,...,...
101,1694,United Microelectronics Corp.,"The companys objective in establishing this ""...",2020,https://www.umc.com/en/Html/UMC_code_of_conduct,NaN,26.0,NaN
102,358,Verizon Communications,Embedding ethics in tech innovation\nBy: Charl...,2021,https://www.verizon.com/about/news/embedding-e...,NaN,20.0,NaN
103,3504,Vodafone,AI & Human Rights: The Vodafone perspective\nM...,2020,https://www.vodafone-institut.de/digitising-eu...,NaN,26.0,NaN
104,325,Wipro Limited,Artificial intelligence (AI) promises to chang...,2018,https://www.wipro.com/holmes/wipro-holmes-advi...,1.0,NaN,2.0


In [ ]:
#F=data.loc[data['Affiliation(U=University,F=Firm)'] == 'F']
#F.to_csv("Ethics_firm.csv")

In [ ]:
#U=data.loc[data['Affiliation(U=University,F=Firm)'] == 'U']
#U.to_csv("Ethics_uni.csv")

In [ ]:
# clean patent. strip, drop nan and keep patents that have more than 5 words.

patent["Principles"] = patent["Principles"].str.strip()
patent = patent[patent["Principles"].isnull() == False]
patent = patent[patent["Principles"].str.count(" ") > 5]
patent = patent.drop_duplicates()

In [ ]:
patent

,CID,Company,Principles,Year,Source,TotalAIEthicsPub,TotalAIpub,AbstractScore
0,291,3M,David Frazee is vice president of 3Ms CRSL gr...,2022,https://futures.3m.com/Artificial-Intelligence...,NaN,82.0,NaN
1,67,Accenture,We welcome steps already being taken in this s...,2019,http://www.ethicscodescollection.org/detail/ae...,8.0,250.0,0.0
2,1451,Adobe,Adobe is taking a deliberate and thoughtful ap...,2022,AI Ethics (adobe.com),2.0,927.0,2.0
3,1693,Amazon,"Responsible Use of Machine Learning \nAt AWS, ...",2022,https://aws.amazon.com/machine-learning/respon...,21.0,710.0,0.0
4,234,American Express,Artificial Intelligence (AI) offers companies ...,2018,https://www.americanexpress.com/en-us/business...,NaN,7.0,NaN
...,...,...,...,...,...,...,...,...
101,1694,United Microelectronics Corp.,"The companys objective in establishing this ""...",2020,https://www.umc.com/en/Html/UMC_code_of_conduct,NaN,26.0,NaN
102,358,Verizon Communications,Embedding ethics in tech innovation\nBy: Charl...,2021,https://www.verizon.com/about/news/embedding-e...,NaN,20.0,NaN
103,3504,Vodafone,AI & Human Rights: The Vodafone perspective\nM...,2020,https://www.vodafone-institut.de/digitising-eu...,NaN,26.0,NaN
104,325,Wipro Limited,Artificial intelligence (AI) promises to chang...,2018,https://www.wipro.com/holmes/wipro-holmes-advi...,1.0,NaN,2.0


In [ ]:
import pickle
from six import iteritems
from collections import Counter
import pandas as pd
import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex
from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER, CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
import os
from gensim.utils import simple_preprocess
from gensim import corpora, models
import re
import nltk
from nltk.corpus import wordnet, stopwords
import pickle
import gensim
from nltk.stem.wordnet import WordNetLemmatizer
from collections import defaultdict
from tqdm import tqdm

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
class Corpus:
    def __init__(self, name, texts=None):
        self.corpus_name = name
        self.num_docs = 0
        self.texts = texts if texts != None else []
        self.dictionary = None
        self.doc_term_mat = None
        self.num_elem = 0

    def setDocTermMatrix(self):
        print("Creating Document Term-Frequency Matrix....")
        self.dictionary = corpora.Dictionary(self.texts)
        #self.dictionary.filter_extremes(no_below=.01, no_above=0.99)
        self.doc_term_mat = [self.dictionary.doc2bow(text) for text in self.texts]
        print("Complete.")

    def updateTexts(self, text):
        self.texts.append(text)
        return self


def setVocab(c):
    print("setting vocab file....")
    word_id = c.dictionary.token2id
    word_id = sorted(word_id.items(), key=lambda x: x[1])
    path = "C:/Users/Admin/Desktop/"+c.corpus_name+".txt"

    with open(path, "w") as f:
        for pair in word_id:
            f.write(pair[0] + "\n")
    return f        


def createLDA_C_File(c):
    print("Creating .dat file....")
    path = "C:/Users/Admin/Desktop/" + c.corpus_name + ".dat"

    with open(path, "w+") as f:
        for i in range(c.num_docs):
            f.write(str(len(c.doc_term_mat[i])) + " ")
            for j in range(len(c.doc_term_mat[i])):
                f.write(str(c.doc_term_mat[i][j][0]) + ":" + str(c.doc_term_mat[i][j][1]) + " ")
            f.write("\n")


def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True, min_len=2))
# deacc: Remove letter accents from the given string
# deaccent("Šéf chomutovských komunistů dostal poštou bílý prášek")
# u"Sef chomutovskych komunistu dostal postou bily prasek"


stop = stopwords.words("english") + ["also", "use", "respective", "respectively",
                                     "one", "may", "two", "within", "say", "wherein", "andor", "thereof", "eg", "etc", "along"]


def remove_stopwords(texts):
    return [[word for word in doc if word not in stop] for doc in tqdm(texts)]


def clean_token(texts):
    newtext = [[word.strip("-") for word in doc if (word.isspace() == False) & (
        word.strip("-").isspace() == False) & (len(word.strip("-")) > 2)] for doc in tqdm(texts)]

    newtext = [[word for word in doc if not (word.isdigit())] for doc in tqdm(newtext)]
    return newtext


# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
def custom_tokenizer(nlp):
    infixes = (
        LIST_ELLIPSES
        + LIST_ICONS
        + [
            r"(?<=[0-9])[+\-\*^](?=[0-9-])",
            r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
                al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
            ),
            r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
            #r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
            r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
        ]
    )

    infix_re = compile_infix_regex(infixes)

    return Tokenizer(nlp.vocab, prefix_search=nlp.tokenizer.prefix_search,
                     suffix_search=nlp.tokenizer.suffix_search,
                     infix_finditer=infix_re.finditer,
                     token_match=nlp.tokenizer.token_match,
                     rules=nlp.Defaults.tokenizer_exceptions)


nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
nlp.tokenizer = custom_tokenizer(nlp)



def lemmatization(texts):
    texts_out = []
    for sent in tqdm(texts):
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc])
    return texts_out


def parseData(inputdir):
    allyear = pd.DataFrame()
    for filename in sorted(os.listdir(inputdir)):
        if ".csv" not in filename:
            continue
        print(filename)
        df = pd.read_csv(inputdir+filename)
        allyear = allyear.append(df)
        # raw_input()
    c = Corpus("all_years")

    allyear["text"] = allyear["text"].str.lower()
    data = allyear["text"].values.tolist()
    data = [re.sub(r"[^-0-9a-zA-Z ]", "", sent) for sent in data]
    print("tokenize")
    data_words = list(sent_to_words(data))
    # print(data_words[:1])
    # len(data)
    print("load bigrams")
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=10)  # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    # print(bigram_mod[data_words[0]])

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in tqdm(texts)]

 

    
    
    



In [ ]:
#print(f)

In [ ]:
allyear = pd.DataFrame()
allyear = allyear.append(patent)
 # raw_input()
c = Corpus("all_years")

allyear 

,CID,Company,Principles,Year,Source,TotalAIEthicsPub,TotalAIpub,AbstractScore
0,291,3M,David Frazee is vice president of 3Ms CRSL gr...,2022,https://futures.3m.com/Artificial-Intelligence...,NaN,82.0,NaN
1,67,Accenture,We welcome steps already being taken in this s...,2019,http://www.ethicscodescollection.org/detail/ae...,8.0,250.0,0.0
2,1451,Adobe,Adobe is taking a deliberate and thoughtful ap...,2022,AI Ethics (adobe.com),2.0,927.0,2.0
3,1693,Amazon,"Responsible Use of Machine Learning \nAt AWS, ...",2022,https://aws.amazon.com/machine-learning/respon...,21.0,710.0,0.0
4,234,American Express,Artificial Intelligence (AI) offers companies ...,2018,https://www.americanexpress.com/en-us/business...,NaN,7.0,NaN
...,...,...,...,...,...,...,...,...
101,1694,United Microelectronics Corp.,"The companys objective in establishing this ""...",2020,https://www.umc.com/en/Html/UMC_code_of_conduct,NaN,26.0,NaN
102,358,Verizon Communications,Embedding ethics in tech innovation\nBy: Charl...,2021,https://www.verizon.com/about/news/embedding-e...,NaN,20.0,NaN
103,3504,Vodafone,AI & Human Rights: The Vodafone perspective\nM...,2020,https://www.vodafone-institut.de/digitising-eu...,NaN,26.0,NaN
104,325,Wipro Limited,Artificial intelligence (AI) promises to chang...,2018,https://www.wipro.com/holmes/wipro-holmes-advi...,1.0,NaN,2.0


In [ ]:
allyear.rename(columns = {'Principles':'Abstract'}, inplace = True)
allyear

,CID,Company,Abstract,Year,Source,TotalAIEthicsPub,TotalAIpub,AbstractScore
0,291,3M,David Frazee is vice president of 3Ms CRSL gr...,2022,https://futures.3m.com/Artificial-Intelligence...,NaN,82.0,NaN
1,67,Accenture,We welcome steps already being taken in this s...,2019,http://www.ethicscodescollection.org/detail/ae...,8.0,250.0,0.0
2,1451,Adobe,Adobe is taking a deliberate and thoughtful ap...,2022,AI Ethics (adobe.com),2.0,927.0,2.0
3,1693,Amazon,"Responsible Use of Machine Learning \nAt AWS, ...",2022,https://aws.amazon.com/machine-learning/respon...,21.0,710.0,0.0
4,234,American Express,Artificial Intelligence (AI) offers companies ...,2018,https://www.americanexpress.com/en-us/business...,NaN,7.0,NaN
...,...,...,...,...,...,...,...,...
101,1694,United Microelectronics Corp.,"The companys objective in establishing this ""...",2020,https://www.umc.com/en/Html/UMC_code_of_conduct,NaN,26.0,NaN
102,358,Verizon Communications,Embedding ethics in tech innovation\nBy: Charl...,2021,https://www.verizon.com/about/news/embedding-e...,NaN,20.0,NaN
103,3504,Vodafone,AI & Human Rights: The Vodafone perspective\nM...,2020,https://www.vodafone-institut.de/digitising-eu...,NaN,26.0,NaN
104,325,Wipro Limited,Artificial intelligence (AI) promises to chang...,2018,https://www.wipro.com/holmes/wipro-holmes-advi...,1.0,NaN,2.0


In [ ]:

allyear["Abstract"] = allyear["Abstract"].str.lower()
data = allyear["Abstract"].values.tolist()
data = [re.sub(r"[^-0-9a-zA-Z ]", "", sent) for sent in data]
print("tokenize")
data_words = list(sent_to_words(data))
    # print(data_words[:1])
    # len(data)
print("load bigrams")
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=10)  # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)
    # print(bigram_mod[data_words[0]])

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in tqdm(texts)]

print("drop stopwords")
data_words_nostops = remove_stopwords(data_words)
print("create bigrams")
data_words_bigrams = make_bigrams(data_words_nostops)
print("lemmatize")
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

print("clean token")
data_words_cleaned = clean_token(data_lemmatized)

   


tokenize
load bigrams
drop stopwords


100%|██████████| 106/106 [00:00<00:00, 368.78it/s]


create bigrams


100%|██████████| 106/106 [00:00<00:00, 560.67it/s]


lemmatize


100%|██████████| 106/106 [00:06<00:00, 16.99it/s]


clean token


100%|██████████| 106/106 [00:00<00:00, 15343.07it/s]


In [ ]:
#from scripts.newsgrp_preprocess import run_preprocess


In [ ]:
#!pip install tomotopy


#For the dataset, I restricted the model to using only words that appeared in at least 30 documents with the min_cf argument while excluding the 10 most frequent words withrm_top. Similarly, I set the concentration parameters gamma=1 and alpha=0.1 given I assume documents share many topics while individual documents only talk about few topics. I initialized the number of topics withinitial_k=3 which acts as a sort of prior. 

In [ ]:
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import sys

In [ ]:
def train_HDPmodel(hdp, word_list, mcmc_iter, burn_in=100, quiet=False):
    '''Wrapper function to train tomotopy HDP Model object
    
    *** Inputs**
    hdp: obj -> initialized HDPModel model
    word_list: list -> lemmatized word list of lists
    mcmc_iter : int -> number of iterations to train the model
    burn_in: int -> MC burn in iterations
    quiet: bool -> flag whether to print iteration LL and Topics, if True nothing prints out
    
    ** Returns**
    hdp: trained HDP Model 
    '''
    
    # Add docs to train
    for vec in word_list:
        hdp.add_doc(vec)

    # Initiate MCMC burn-in 
    hdp.burn_in = 100
    hdp.train(0)
    print('Num docs:', len(hdp.docs), ', Vocab size:', hdp.num_vocabs, ', Num words:', hdp.num_words)
    print('Removed top words:', hdp.removed_top_words)
    print('Training...', file=sys.stderr, flush=True)

    # Train model
    
    step=round(mcmc_iter*0.10)
    for i in range(0, mcmc_iter, step):
        hdp.train(step, workers=3)
        if not quiet:
            print('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, hdp.ll_per_word, hdp.live_k))
        
    print("Done\n")  
    
    return hdp

In [ ]:
#!pip install tomotopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.5 MB 4.3 MB/s 


In [ ]:
import tomotopy as tp
term_weight = tp.TermWeight.ONE
hdp = tp.HDPModel(tw=term_weight, min_cf=10, rm_top=10, gamma=1,
                  alpha=0.1, initial_k=7, seed=99999)

In [ ]:
# Add docs to train
for vec in data_words_cleaned:
    hdp.add_doc(vec)


In [ ]:
# Initiate sampling burn-in  (i.e. discard N first iterations)
hdp.burn_in = 50
hdp.train(0)
print('Num docs:', len(hdp.docs), ', Vocab size:', hdp.num_vocabs,
      ', Num words:', hdp.num_words)
print('Removed top words:', hdp.removed_top_words)


Num docs: 106 , Vocab size: 1176 , Num words: 42530
Removed top words: ['datum', 'system', 'use', 'model', 'technology', 'company', 'principle', 'ethic', 'human', 'ethical']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:

# Train model
for i in range(0, 500, 50):
    hdp.train(50) # 100 iterations at a time
    print('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, hdp.ll_per_word, hdp.live_k))
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration: 0	Log-likelihood: -6.890987901620652	Num. of topics: 12
Iteration: 50	Log-likelihood: -6.90383576857489	Num. of topics: 9
Iteration: 100	Log-likelihood: -6.894362929866657	Num. of topics: 10
Iteration: 150	Log-likelihood: -6.895193063703948	Num. of topics: 9
Iteration: 200	Log-likelihood: -6.873821216139795	Num. of topics: 9
Iteration: 250	Log-likelihood: -6.871879634590868	Num. of topics: 9
Iteration: 300	Log-likelihood: -6.8691919331282625	Num. of topics: 9
Iteration: 350	Log-likelihood: -6.860216818064635	Num. of topics: 9
Iteration: 400	Log-likelihood: -6.859272432635908	Num. of topics: 9
Iteration: 450	Log-likelihood: -6.857805350482791	Num. of topics: 10


In [ ]:
def get_hdp_topics(hdp, top_n=15):
    '''Wrapper function to extract topics from trained tomotopy HDP model 
    
    ** Inputs **
    hdp:obj -> HDPModel trained model
    top_n: int -> top n words in topic based on frequencies
    
    ** Returns **
    topics: dict -> per topic, an arrays with top words and associated frequencies 
    '''
    
    # Get most important topics by # of times they were assigned (i.e. counts)
    sorted_topics = [k for k, v in sorted(enumerate(hdp.get_count_by_topics()), key=lambda x:x[1], reverse=True)]

    topics=dict()
    
    # For topics found, extract only those that are still assigned
    for k in sorted_topics:
        if not hdp.is_live_topic(k): continue # remove un-assigned topics at the end (i.e. not alive)
        topic_wp =[]
        for word, prob in hdp.get_topic_words(k, top_n=top_n):
            topic_wp.append((word, prob))

        topics[k] = topic_wp # store topic word/frequency array
        
    return topics

In [ ]:
topics = get_hdp_topics(hdp, top_n=10) 

In [ ]:
topics

{4: [('need', 0.007701963651925325),
  ('bias', 0.006787205580621958),
  ('work', 0.006639664061367512),
  ('people', 0.006492122542113066),
  ('make', 0.006403597537428141),
  ('well', 0.006049497984349728),
  ('develop', 0.0058134314604103565),
  ('process', 0.005577364936470985),
  ('help', 0.00548883993178606),
  ('decision', 0.0054593319073319435)],
 5: [('user', 0.014007819816470146),
  ('ensure', 0.013150372542440891),
  ('activity', 0.010292217135429382),
  ('digital_solution', 0.009720586240291595),
  ('process', 0.009434770792722702),
  ('service', 0.009434770792722702),
  ('provide', 0.009148955345153809),
  ('respect', 0.008863139897584915),
  ('issue', 0.008577324450016022),
  ('risk', 0.008577324450016022)],
 0: [('worker', 0.030846111476421356),
  ('supplier', 0.02664077840745449),
  ('provide', 0.019631892442703247),
  ('include', 0.018931005150079727),
  ('shall', 0.018230115994811058),
  ('prediction', 0.01682833768427372),
  ('flexibility', 0.014024783857166767),
  (